In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
serif = False
render_format = "pdf"
if serif:
    dir_postfix = ""
    plt.rcParams["font.family"] = "serif"
else:
    dir_postfix = "sans"
    plt.rcParams["font.family"] = "Liberation Sans"
plt.rcParams["font.size"] = 10
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["ps.fonttype"] = 42

from results import *

# Real-World Analysis: Push Chair

In [2]:
real_exp_keys = []

# Generate temporal consistency experiment keys.
real_exp_keys += get_temporal_consistency_exp_keys(
    pred_horizons=[16],
    sample_sizes=[256],
    error_fns=["mmd_rbf_all_median", "kde_kl_all_rev_eig", "kde_kl_all_for_eig", "mse_all"],
    aggr_fns=["min"],
)

# Generate ensemble experiment keys.
real_exp_keys += get_ensemble_exp_keys(
    pred_horizons=[16],
    sample_sizes=[256],
    action_spaces=["all"],
)

# Generate VLM experiment keys.
real_exp_keys += get_vlm_exp_keys(
    models=[
        "gpt-4o", 
        "claude-3-5-sonnet-20240620", 
        "gemini-1-5-pro"
    ],
    templates={
        "gpt-4o": ["video_qa"],
        "claude-3-5-sonnet-20240620": ["video_qa"],
        "gemini-1-5-pro": ["video_qa"],
    }
)

# Load results.
real_splits = ["test"]
real_metrics = compile_metrics(
    domain="0914_push_chair_4",
    splits=real_splits,
    exp_keys=real_exp_keys,
    return_test_data=True,
    return_test_frame=True,
)

In [ ]:
for exp_key in real_exp_keys:
    row = f"{exp_key} & "
    
    # Dataset split result.
    for split in real_splits:
        for metric in ["TPR", "TNR", "Accuracy", "TP Time Mean"]:
            stat = real_metrics[split][exp_key]["metrics"][metric]
            if metric == "TP Time Mean":
                stat = stat / 3
            row += f"{round(stat,2):0.2f} & "

    print(row[:-2] + "\\\\")

In [ ]:
# Generate sentinel experiment keys.
stac_exp_keys = get_temporal_consistency_exp_keys(
    pred_horizons=[16],
    sample_sizes=[256],
    error_fns=["kde_kl_all_rev_eig"],
    aggr_fns=[""],
)
vlm_exp_keys = get_vlm_exp_keys(
    models = ["gpt-4o"],
    templates = {"gpt-4o": ["video_qa"]},
)

_ = compute_sentinel_result(
    stac_exp_key=stac_exp_keys[0],
    vlm_exp_keys_list=[vlm_exp_keys],
    splits_list=[["test"]],
    metrics_list=[real_metrics],
    time_mod=3.0,
    domain_names=["Push Chair"]
)